In [1]:
from bytetracker import BYTETracker
import torch
from ultralytics import YOLO
import cv2
import supervision as sv

from tqdm import tqdm
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
current_path = os.getcwd()

In [3]:
# define the model
# version = 'v8'
# model_path = current_path + f'/checkpoints/YOLO/{version}/weights/'
# model = YOLO(model_path+'best.pt')
# model.info()

In [4]:
model = YOLO('yolov8x.pt')

In [6]:
# get the video generator
filename = 'test.mp4'
video_path = current_path + f'/data/videos/{filename}'
frames = sv.get_video_frames_generator(video_path)
video_info = sv.VideoInfo.from_video_path(video_path)

In [7]:
dest_path = current_path + '/data/videos/output.mp4'

In [8]:
# Define the box annotator
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

In [9]:
# Define classes 
classes = model.model.names
classes

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [10]:
video_info.total_frames

1800

In [23]:
generator = sv.get_video_frames_generator(video_path)
iterator = iter(generator)
frame = next(iterator)

cv2.imwrite("frame.jpg", frame)

True

In [11]:
class_ids = [2, 5, 7]

# Initialize the vehicle counts for two polygonal zones
zone_1_count = 0
zone_2_count = 0

# Initialize sets to track vehicles in each zone
zone_1_tracked_ids = set()
zone_2_tracked_ids = set()

# Define the polygonal zones using a list of (x, y) points
zone_1_points = np.array([[1952, 758],[1912, 2150],[524, 2146],[1400, 742]])
zone_2_points = np.array([[1928, 2150],[1956, 750],[2440, 746],[3328, 2150]])

# Create PolygonZone objects
zone_1 = sv.PolygonZone(polygon=zone_1_points)
zone_2 = sv.PolygonZone(polygon=zone_2_points)

zone1_annotator = sv.PolygonZoneAnnotator(zone=zone_1, color=sv.Color.WHITE, thickness=6, text_thickness=6, text_scale=4)
zone2_annotator = sv.PolygonZoneAnnotator(zone=zone_2, color=sv.Color.WHITE, thickness=6, text_thickness=6, text_scale=4)

# Going through the frames
with sv.VideoSink(dest_path, video_info=video_info) as sink:
    for frame in tqdm(frames, total=video_info.total_frames):
        annotated_frame = frame.copy()
        
        # # Resize the frame
        # resized_frame = cv2.resize(frame, (640, 640))
        
        # Get the results from the model
        results = model(frame, verbose=False)[0]

        # Get the detections
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[np.isin(detections.class_id, class_ids)]
        detections = tracker.update_with_detections(detections)
        
        # Draw the zones on the frame
        annotated_frame = sv.draw_polygon(annotated_frame, zone_1_points, color=sv.Color.WHITE, thickness=2)
        annotated_frame = sv.draw_polygon(annotated_frame, zone_2_points, color=sv.Color.WHITE, thickness=2)
        
        labels = [
            f"#{tracker_id} {results.names[class_id]}: {conf:.2f}"
            for class_id, tracker_id, conf
            in zip(detections.class_id, detections.tracker_id, detections.confidence)
        ]
        
        # Process detections for Zone 1
        for i in range(len(detections.xyxy)):
            tracker_id = detections.tracker_id[i]
            if tracker_id not in zone_1_tracked_ids and zone_1.trigger(detections=detections[i:i+1]):
                zone_1_tracked_ids.add(tracker_id)
                zone_1_count += 1

        # Process detections for Zone 2
        for i in range(len(detections.xyxy)):
            tracker_id = detections.tracker_id[i]
            if tracker_id not in zone_2_tracked_ids and zone_2.trigger(detections=detections[i:i+1]):
                zone_2_tracked_ids.add(tracker_id)
                zone_2_count += 1


        # Annotate the frame with bounding boxes and labels
        annotated_frame = box_annotator.annotate(annotated_frame, detections=detections)
        annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
        
        text_anchor = sv.Point(x=1080, y=180)
        text = f"Lane 1: {zone_1_count}"
        annotated_frame = sv.draw_text(
            scene=annotated_frame,
            text=text,
            text_anchor=text_anchor,
            text_color=sv.Color.BLACK,
            background_color=sv.Color.YELLOW,
            text_scale=5,
            text_thickness=6,
            text_font=cv2.FONT_HERSHEY_COMPLEX
        )
        
        text_anchor = sv.Point(x=3080, y=180)
        text = f"Lane 2: {zone_2_count}"
        final_frame = sv.draw_text(
            scene=annotated_frame,
            text=text,
            text_anchor=text_anchor,
            text_color=sv.Color.BLACK,
            background_color=sv.Color.YELLOW,
            text_scale=5,
            text_thickness=6,
            text_font=cv2.FONT_HERSHEY_COMPLEX
        )

        sink.write_frame(final_frame)

100%|██████████| 1800/1800 [12:18<00:00,  2.44it/s]


In [12]:
# Output the total count of vehicles in each zone
print(f"Total number of vehicles in Zone 1: {zone_1_count}")
print(f"Total number of vehicles in Zone 2: {zone_2_count}")

Total number of vehicles in Zone 1: 91
Total number of vehicles in Zone 2: 76


In [17]:
video_info.resolution_wh

(3840, 2160)

In [24]:
# plt.imshow(frame)
# plt.axis('off')
# plt.show()